In [4]:
county_stations = {
    'Tarrant': ['DFW', 'GKY', 'FTW', 'AFW'],
    'McLennan': ['ACT', 'PWG'],
    'Montague': ['0F2'],
    'Stephens': ['BKD'],
    'Wise': ['XBP', 'LUD'],
    'Johnson': ['CPT'],
    'Comanche': ['MKN'],
    'Navarro': ['CRS'],
    'Dallas': ['DAL', 'RBD'],
    'Denton': ['DTO'],
    'Cooke': ['GLE'],
    'Coryell': ['GOP'],
    'Young': ['RPH'],
    'Hood': ['GDJ'],
    'Hunt': ['GVT'],
    'Hamilton': ['MNZ'],
    'Hill': ['INJ'],
    'Collin': ['TKI'],
    'Palo Pinto': ['MWL'],
    'Grayson': ['GYI'],
    'Erath': ['SEP'],
    'Kaufman': ['TRL'],
    'Ellis': ['JWY'],
    'Eastland': ['BKD', 'MKN'],
    'Jack': ['RPH', 'XBP', 'LUD'],
    'Parker': ['MWL', 'GDJ', 'DFW', 'GKY', 'FTW', 'AFW', 'DTO'],
    'Rockwall': ['DAL', 'RBD', 'TKI', 'GVT', 'TRL'],
    'Somervell': ['SEP', 'GDJ', 'CPT'],
    'Bosque': ['MNZ', 'INJ', 'ACT', 'PWG', 'GOP']
}

station_counties = {}
for county, stations in county_stations.items():
    for station in stations:
        if station not in station_counties:
            station_counties[station] = []
        station_counties[station].append(county)
station_counties

{'DFW': ['Tarrant', 'Parker'],
 'GKY': ['Tarrant', 'Parker'],
 'FTW': ['Tarrant', 'Parker'],
 'AFW': ['Tarrant', 'Parker'],
 'ACT': ['McLennan', 'Bosque'],
 'PWG': ['McLennan', 'Bosque'],
 '0F2': ['Montague'],
 'BKD': ['Stephens', 'Eastland'],
 'XBP': ['Wise', 'Jack'],
 'LUD': ['Wise', 'Jack'],
 'CPT': ['Johnson', 'Somervell'],
 'MKN': ['Comanche', 'Eastland'],
 'CRS': ['Navarro'],
 'DAL': ['Dallas', 'Rockwall'],
 'RBD': ['Dallas', 'Rockwall'],
 'DTO': ['Denton', 'Parker'],
 'GLE': ['Cooke'],
 'GOP': ['Coryell', 'Bosque'],
 'RPH': ['Young', 'Jack'],
 'GDJ': ['Hood', 'Parker', 'Somervell'],
 'GVT': ['Hunt', 'Rockwall'],
 'MNZ': ['Hamilton', 'Bosque'],
 'INJ': ['Hill', 'Bosque'],
 'TKI': ['Collin', 'Rockwall'],
 'MWL': ['Palo Pinto', 'Parker'],
 'GYI': ['Grayson'],
 'SEP': ['Erath', 'Somervell'],
 'TRL': ['Kaufman', 'Rockwall'],
 'JWY': ['Ellis']}

In [18]:
import pandas as pd

dates = pd.read_csv('dates.csv')
dates['Date'] = pd.to_datetime(dates['Date'])

In [24]:
import pandas as pd
import os

selected_dfs = []

pd.options.mode.chained_assignment = None

stations_dir = r"..\data"
for file in os.listdir(stations_dir):
    station = file[:3]

    if station not in station_counties:
        continue

    print(station)

    path = os.path.join(stations_dir, file)
    station_df = pd.read_csv(path)
    expanded_df = pd.DataFrame()

    #print(station_df.head(3))

    selected = station_df[['Date', 'Avg Temp', 'Max Temp', 'Min Temp', 'Total Precip', 'Total Snow', 'Avg Wind Speed', 'Max Wind Speed']]

    selected_cols = selected.columns.drop('Date')
    selected[selected_cols] = selected[selected_cols].apply(pd.to_numeric, errors='coerce')
    selected['Date'] = pd.to_datetime(selected['Date'])

    selected = selected[selected['Max Wind Speed'] < 100] # correct for some error where the max wind speed is swapped with the max wind direction
    selected.dropna(subset=['Avg Temp', 'Max Temp', 'Min Temp'], inplace=True) # rows where these are NaN are likely missing all values anyway
    selected.fillna(0, inplace=True)

    # just get the specific range of dates from dates.csv, make sure no gaps exist
    selected = pd.merge(dates, selected, on='Date', how='left')
    selected.interpolate(method='pad', inplace=True)
    
    daily_rows = []
    
    for _, row in selected.iterrows():
        date = row['Date']
        max_t = row['Max Temp']
        min_t = row['Min Temp']
        avg_t = row['Avg Temp']

        extra_t = 2*avg_t - (max_t + min_t) / 2

        precip = row['Total Precip'] / 4
        snow = row['Total Snow'] / 4

        avg_w = row['Avg Wind Speed']

        daily_rows.append({'Date': date, 'Hour': 6, 'Temperature': min_t, 'Precipitation': precip, 'Snow': snow, 'Wind Speed': avg_w})
        daily_rows.append({'Date': date, 'Hour': 12, 'Temperature': extra_t, 'Precipitation': precip, 'Snow': snow, 'Wind Speed': avg_w})
        daily_rows.append({'Date': date, 'Hour': 18, 'Temperature': max_t, 'Precipitation': precip, 'Snow': snow, 'Wind Speed': avg_w})
        daily_rows.append({'Date': date, 'Hour': 24, 'Temperature': extra_t, 'Precipitation': precip, 'Snow': snow, 'Wind Speed': avg_w})
        
    daily_df = pd.DataFrame(daily_rows)
    
    daily_df.to_csv(f'data\\{station}_hourly.csv', index=False)

0F2


C:\Users\Austin\AppData\Local\Temp\ipykernel_43220\3091695351.py:35: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selected.interpolate(method='pad', inplace=True)


ACT


C:\Users\Austin\AppData\Local\Temp\ipykernel_43220\3091695351.py:35: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selected.interpolate(method='pad', inplace=True)


AFW


...

C:\Users\Austin\AppData\Local\Temp\ipykernel_43220\3091695351.py:35: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selected.interpolate(method='pad', inplace=True)


TRL


C:\Users\Austin\AppData\Local\Temp\ipykernel_43220\3091695351.py:35: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selected.interpolate(method='pad', inplace=True)


XBP


C:\Users\Austin\AppData\Local\Temp\ipykernel_43220\3091695351.py:35: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  selected.interpolate(method='pad', inplace=True)
